<a href="https://colab.research.google.com/github/isabelsanchez2-cpu/IA2025/blob/main/04_modelo_con_preprocesado_de_otra_forma_y_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================================
# 04 - MODELO GRADIENT BOOSTING (XGBoost)
# ============================================================================

# %% [markdown]
"""
# 04 - Modelo con Gradient Boosting (XGBoost)

## Estrategia
Este notebook utiliza XGBoost, un algoritmo de gradient boosting que suele dar
excelentes resultados en competencias de Kaggle.

## Diferencias con modelo 99:
- Usa XGBoost en lugar de Random Forest
- Gradient Boosting construye árboles secuencialmente
- Incluye análisis de importancia de features
- Parámetros optimizados para clasificación multiclase
"""

# %% Importar librerías
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

print('✓ Librerías cargadas')

# %% Cargar datos
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(f'Train: {train.shape}')
print(f'Test: {test.shape}')

# %% Distribución de clases
print('\nDistribución de clases:')
print(train['RENDIMIENTO_GLOBAL'].value_counts().sort_index())

# %% Separar componentes
train_ids = train['ID'].copy()
test_ids = test['ID'].copy()
y_train = train['RENDIMIENTO_GLOBAL'].copy()

X_train = train.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1)
X_test = test.drop(['ID'], axis=1)

# %% Identificar columnas
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

print(f'\nNuméricas: {len(numeric_cols)}')
print(f'Categóricas: {len(categorical_cols)}')

# %% Limpieza
print('\n--- Limpieza ---')
for col in categorical_cols:
    X_train[col] = X_train[col].replace(['', ' ', 'nan'], np.nan)
    X_test[col] = X_test[col].replace(['', ' ', 'nan'], np.nan)

# %% Imputación
print('\n--- Imputación ---')
if len(numeric_cols) > 0:
    num_imp = SimpleImputer(strategy='median')
    X_train[numeric_cols] = num_imp.fit_transform(X_train[numeric_cols])
    X_test[numeric_cols] = num_imp.transform(X_test[numeric_cols])

if len(categorical_cols) > 0:
    cat_imp = SimpleImputer(strategy='most_frequent')
    X_train[categorical_cols] = cat_imp.fit_transform(X_train[categorical_cols])
    X_test[categorical_cols] = cat_imp.transform(X_test[categorical_cols])

print('✓ Imputación completada')

# %% Codificación
print('\n--- Codificación ---')
for col in categorical_cols:
    le = LabelEncoder()
    combined = pd.concat([X_train[col].astype(str), X_test[col].astype(str)])
    le.fit(combined)
    X_train[col] = le.transform(X_train[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))

print(f'✓ Codificadas {len(categorical_cols)} columnas')

# %% Limpieza final
X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0)
X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0)

print(f'\n✓ Datos listos')
print(f'  NaN en train: {X_train.isnull().sum().sum()}')
print(f'  NaN en test: {X_test.isnull().sum().sum()}')

# %% Codificar target para XGBoost
print('\n--- Preparando target ---')
le_target = LabelEncoder()
y_train_encoded = le_target.fit_transform(y_train)

print(f'✓ Clases originales: {le_target.classes_}')
print(f'✓ Clases codificadas: {np.unique(y_train_encoded)}')

# %% Entrenamiento XGBoost
print('\n--- Entrenando XGBoost ---')
print('XGBoost es un gradient boosting que construye árboles secuencialmente')

xgb_model = XGBClassifier(
    n_estimators=200,        # Número de árboles
    max_depth=8,             # Profundidad máxima
    learning_rate=0.1,       # Tasa de aprendizaje
    subsample=0.8,           # Proporción de muestras por árbol
    colsample_bytree=0.8,    # Proporción de features por árbol
    objective='multi:softmax',  # Clasificación multiclase
    num_class=4,             # 4 clases
    random_state=42,
    n_jobs=-1,
    eval_metric='mlogloss'
)

print('\nEntrenando modelo...')
xgb_model.fit(X_train, y_train_encoded, verbose=False)
print('✓ XGBoost entrenado')

# %% Evaluación
train_score = xgb_model.score(X_train, y_train_encoded)
print(f'\n✓ Accuracy en train: {train_score:.4f}')

# %% Validación cruzada
print('\n--- Validación Cruzada ---')
cv_scores = cross_val_score(xgb_model, X_train, y_train_encoded, cv=3, n_jobs=-1)
print(f'CV Scores: {cv_scores.round(4)}')
print(f'✓ CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})')

# %% Feature Importance
print('\n--- Feature Importance (Top 15) ---')
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance.head(15).to_string(index=False))

# %% Predicciones
print('\n--- Predicciones ---')
predictions_encoded = xgb_model.predict(X_test)

# Decodificar predicciones
predictions = le_target.inverse_transform(predictions_encoded)
print(f'✓ Predicciones: {len(predictions)}')

print('\nDistribución de predicciones:')
pred_counts = pd.Series(predictions).value_counts().sort_index()
print(pred_counts)
print('\nPorcentaje:')
print((pred_counts / len(predictions) * 100).round(2))

# %% Submission
submission = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': predictions
})

print(f'\n✓ Submission shape: {submission.shape}')
print(f'✓ IDs únicos: {submission["ID"].nunique()}')
print(f'✓ Clases predichas: {sorted(submission["RENDIMIENTO_GLOBAL"].unique())}')

print('\nPrimeras filas:')
print(submission.head(10))

# %% Guardar
submission.to_csv('submission_xgboost.csv', index=False)
print('\n' + '='*60)
print('✓✓✓ Archivo submission_xgboost.csv generado ✓✓✓')
print('='*60)
print('\nEste es un modelo alternativo usando XGBoost')

# %% [markdown]
"""
## Resumen Modelo 04

### XGBoost (Gradient Boosting)
- **Tipo**: Gradient Boosting Machine
- **Árboles**: 200 (secuenciales, no paralelos)
- **Learning rate**: 0.1
- **Max depth**: 8

### Ventajas de XGBoost
- ✅ Muy efectivo en competencias Kaggle
- ✅ Maneja bien features categóricas codificadas
- ✅ Regularización incorporada
- ✅ Feature importance integrado

### Diferencias con otros modelos

| Característica | RF (99) | SVM (03) | XGBoost (04) |
|----------------|---------|----------|--------------|
| Construcción | Paralela | N/A | Secuencial |
| Velocidad | Rápido | Lento | Media |
| Overfitting | Bajo | Medio | Controlable |
| Interpretabilidad | Alta | Baja | Alta |

### Conclusión
XGBoost es una excelente alternativa que frecuentemente da buenos
resultados en problemas de clasificación tabulares.
"""

✓ Librerías cargadas
Train: (692500, 21)
Test: (296786, 20)

Distribución de clases:
RENDIMIENTO_GLOBAL
alto          175619
bajo          172987
medio-alto    171619
medio-bajo    172275
Name: count, dtype: int64

Numéricas: 5
Categóricas: 14

--- Limpieza ---

--- Imputación ---
✓ Imputación completada

--- Codificación ---
✓ Codificadas 14 columnas

✓ Datos listos
  NaN en train: 0
  NaN en test: 0

--- Preparando target ---
✓ Clases originales: ['alto' 'bajo' 'medio-alto' 'medio-bajo']
✓ Clases codificadas: [0 1 2 3]

--- Entrenando XGBoost ---
XGBoost es un gradient boosting que construye árboles secuencialmente

Entrenando modelo...
✓ XGBoost entrenado

✓ Accuracy en train: 0.4902

--- Validación Cruzada ---
CV Scores: [0.4323 0.4326 0.4331]
✓ CV Accuracy: 0.4327 (+/- 0.0004)

--- Feature Importance (Top 15) ---
                    feature  importance
E_VALORMATRICULAUNIVERSIDAD    0.177893
          F_ESTRATOVIVIENDA    0.118196
      E_PAGOMATRICULAPROPIO    0.099557
          

'\n## Resumen Modelo 04\n\n### XGBoost (Gradient Boosting)\n- **Tipo**: Gradient Boosting Machine\n- **Árboles**: 200 (secuenciales, no paralelos)\n- **Learning rate**: 0.1\n- **Max depth**: 8\n\n### Ventajas de XGBoost\n- ✅ Muy efectivo en competencias Kaggle\n- ✅ Maneja bien features categóricas codificadas\n- ✅ Regularización incorporada\n- ✅ Feature importance integrado\n\n### Diferencias con otros modelos\n\n| Característica | RF (99) | SVM (03) | XGBoost (04) |\n|----------------|---------|----------|--------------|\n| Construcción | Paralela | N/A | Secuencial |\n| Velocidad | Rápido | Lento | Media |\n| Overfitting | Bajo | Medio | Controlable |\n| Interpretabilidad | Alta | Baja | Alta |\n\n### Conclusión\nXGBoost es una excelente alternativa que frecuentemente da buenos\nresultados en problemas de clasificación tabulares.\n'